## 13.1.1 加载数据和链式转换

In [2]:
import tensorflow as tf
X= tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

C:\Users\Administrator\anaconda3\envs\DeepLearning-py310\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [3]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


## 13.1.2 链式转换

In [4]:
# 将数据原封不动的复制三份 然后按顺序分为7组 最后一组可以使用drop_remainder=True来删除 如果大小不一样的话
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [7]:
# 可以指定num_parallel_calls来指定使用多少个线程对每个元素进行并行处理
dataset = dataset.map(lambda x: x * 2,num_parallel_calls= tf.data.experimental.AUTOTUNE)
print([item for item in dataset],end='           ')

[<tf.Tensor: shape=(7,), dtype=int32, numpy=array([ 0,  8, 16, 24, 32, 40, 48])>, <tf.Tensor: shape=(7,), dtype=int32, numpy=array([56, 64, 72,  0,  8, 16, 24])>, <tf.Tensor: shape=(7,), dtype=int32, numpy=array([32, 40, 48, 56, 64, 72,  0])>, <tf.Tensor: shape=(7,), dtype=int32, numpy=array([ 8, 16, 24, 32, 40, 48, 56])>, <tf.Tensor: shape=(2,), dtype=int32, numpy=array([64, 72])>]           

In [8]:
# map应用于每个元素, apply应用于整个数据集
dataset = dataset.apply(tf.data.experimental.unbatch())
for item in dataset:
    print(item)

Instructions for updating:
Use `tf.data.Dataset.unbatch()`.
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(24, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(48, shape=(), dtype=int32)
tf.Tensor(56, shape=(), dtype=int32)
tf.Tensor(64, shape=(), dtype=int32)
tf.Tensor(72, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(24, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(40, shape=(), dtype=int32)
tf.Tensor(48, shape=(), dtype=int32)
tf.Tensor(56, shape=(), dtype=int32)
tf.Tensor(64, shape=(), dtype=int32)
tf.Tensor(72, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(24, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(40, shape=(

In [9]:
# 使用filter函数来过滤数据
dataset = dataset.filter(lambda x: x < 10)
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)


## 13.1.3 乱序数据

In [11]:
dataset = tf.data.Dataset.range(10).repeat(3)
# 如果在乱序后数据集上面调用repeat则可以将reshuffle_each_iteration设置为False 这样repeat每次会生成一样的数据
dataset = dataset.shuffle(buffer_size=5, seed= 42).batch(7, drop_remainder=True)
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)


### 如果数据集太大的话可以考虑直接打乱整个数据集 或者将文件分成几个部分 每次随机采用